In [19]:
# Import data
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
transaction = pd.read_csv("https://s3.amazonaws.com/pernalonga/transaction_table.csv",header=0)
product = pd.read_csv("https://s3.amazonaws.com/pernalonga/product_table.csv",header=0)

In [20]:
# Delete wrong records and convert date to datetime object
transaction=transaction[transaction['tran_prod_paid_amt']>0]
transaction['tran_dt'] = pd.to_datetime(transaction['tran_dt'], format='%Y-%m-%d')
# Just use 2017 reords to find substitute and complement
transaction['year'] = transaction['tran_dt'].dt.year
transaction=transaction[transaction['year']==2017]

In [22]:
# Create a new column with new transaction_id
transaction['id'] = transaction['tran_dt'].dt.strftime('%Y%m%d') + transaction['cust_id'].astype(str) \
+ transaction['store_id'].astype(str)

In [208]:
transaction.head()

,cust_id,tran_id,tran_dt,store_id,prod_id,prod_unit,tran_prod_sale_amt,tran_prod_sale_qty,tran_prod_discount_amt,tran_prod_offer_cts,tran_prod_paid_amt,prod_unit_price,year,id
0,139662,2.017110e+18,2017-11-03,584,145519008,CT,2.89,4.0,0.00,0,2.89,0.7225,2017,20171103139662584
1,799924,2.017111e+18,2017-11-12,349,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,2017,20171112799924349
2,1399898,2.017102e+18,2017-10-21,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,2017,201710211399898684
3,1399898,2.017111e+18,2017-11-11,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,2017,201711111399898684
4,1399898,2.017121e+18,2017-12-05,684,145519008,CT,2.89,4.0,-1.45,1,1.44,0.7225,2017,201712051399898684


In [209]:
# Get cleaned data with unique transaction id and product id
data=transaction.iloc[:,[13,4]]

In [210]:
data.head()

,id,prod_id
0,20171103139662584,145519008
1,20171112799924349,145519008
2,201710211399898684,145519008
3,201711111399898684,145519008
4,201712051399898684,145519008


In [211]:
# Number of transactions
data.shape[0]

15276448

In [212]:
# Extract products which contribute 90% sales
product2017_sales = transaction[['prod_id','tran_prod_paid_amt']].groupby('prod_id').sum().reset_index().\
sort_values('tran_prod_paid_amt',ascending=False)
product2017_sales['percentage']=product2017_sales['tran_prod_paid_amt']/33202391.5*100
product2017_sales['percentage'][0:4800].sum()

90.2324118429834

In [272]:
# Prepare a new dataset for complement loop later
com = pd.DataFrame(product2017_sales['prod_id'][0:4800])
com['complement']=0

In [ ]:
# Find the complement for product selected above, which should be bought together the most
from collections import Counter
productlist=list(product2017_sales['prod_id'][0:4800])
i=0
for prod_id in productlist:
    tran = list(set(data[data['prod_id']==prod_id]['id']))
    a = data[data['id'].isin(tran)]
    b = a[a['prod_id']!=prod_id]
    counts=Counter(list(b['prod_id']))
    com.iloc[i,1]=counts.most_common(1)[0][0]
    i=i+1

In [313]:
# Output complement file
com.to_csv('complement.csv')